In [4]:
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [11]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

Episode = namedtuple('Episode', field_names=['reward', 'steps'])
Step = namedtuple('Step', field_names=['observation', 'action'])

In [8]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )
        
    def forward(self, x):
        return self.net(x)

In [13]:
def iterate_batches(env, net, batch_size):
    
    def init_episode(env):
        episode_reward = 0.0
        steps = []
        obs = env.reset()
        return episode_reward, steps, obs
    
    batch = []
    episode_reward, steps, obs = init_episode(env)
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        steps.append(Step(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=steps))
            episode_reward, steps, next_obs = init_episode(env)
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [31]:
def filter_batch(batch, percentile):
    rewards = [ episode.reward for episode in batch ]
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))
    train_obs = []
    train_act = []
    for episode in batch:
        if episode.reward < reward_bound:
            continue
        train_obs += [ step.observation for step in episode.steps ]
        train_act += [ step.action for step in episode.steps ]
    
    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [33]:
env = gym.make("CartPole-v0")
env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-cartpole")

for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 199:
        print("Solved!")
        break
writer.close

0: loss=0.677, reward_mean=19.8, reward_bound=20.0
1: loss=0.662, reward_mean=17.8, reward_bound=17.5
2: loss=0.680, reward_mean=22.6, reward_bound=29.0
3: loss=0.663, reward_mean=23.8, reward_bound=19.0
4: loss=0.663, reward_mean=33.4, reward_bound=33.0
5: loss=0.661, reward_mean=32.1, reward_bound=34.5
6: loss=0.642, reward_mean=32.2, reward_bound=34.0
7: loss=0.638, reward_mean=43.2, reward_bound=54.5
8: loss=0.625, reward_mean=44.4, reward_bound=50.0
9: loss=0.615, reward_mean=49.0, reward_bound=51.5
10: loss=0.618, reward_mean=53.4, reward_bound=52.5
11: loss=0.605, reward_mean=59.9, reward_bound=66.0
12: loss=0.582, reward_mean=63.1, reward_bound=79.0
13: loss=0.597, reward_mean=53.0, reward_bound=53.0
14: loss=0.578, reward_mean=64.6, reward_bound=67.5
15: loss=0.572, reward_mean=77.8, reward_bound=87.5
16: loss=0.571, reward_mean=85.8, reward_bound=91.5
17: loss=0.566, reward_mean=92.8, reward_bound=120.5
18: loss=0.567, reward_mean=105.6, reward_bound=132.0
19: loss=0.579, rew

<bound method SummaryWriter.close of <tensorboardX.writer.SummaryWriter object at 0x7fe30ea46780>>